In [2]:
# nltk.download('stopwords')
# nltk.download('wordnet')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
import it_core_news_sm
import re
from itertools import combinations
import itertools
import os
import networkx as nx

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
word_tokenize = WhitespaceTokenizer()
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud

In [2]:
database_words = pd.read_csv("~/Scrivania/common_start/database/id_lemmas.csv",index_col = 0 , sep=',', engine='python')
database_words['text_nlp'] = database_words.apply(lambda row: word_tokenize.tokenize(row['text_nlp']), axis = 1)
database_words

,text_nlp
0,"[diro, gente, guerra, portato, casa, rimpatriare]"
1,"[marcello, perfavore]"
2,"[patrio, patriota, difeso, proprio, radice, so..."
3,"[musulmano, comandare, casa]"
4,"[odiare, dipendere, odiare, comandare, librare]"
...,...
78170,"[vedere, ruspa, inviato, salvini, ripulire, am..."
78171,"[fino, sentire, sempre, richiamare, africo, pa..."
78172,"[sparare, ladro, occhio, penna, barbetta]"
78173,"[consigliatissima, africo]"


In [3]:
database_comments = pd.read_csv("~/Scrivania/common_start/database/com_liwc.csv", sep='\t', engine='python')
database_comments.head(2)

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,...,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## obtain the subset

In [4]:
subset_comments = database_comments.loc[:,['c_rating','p_dx_sx']]
dx_sx = subset_comments.p_dx_sx.unique().tolist()

## obtain the file with nodes+attributes and edges

In [6]:
def get_id_from_node(word, ids):
    return ids.index[ids.label == word][0]

def get_subset_dxsx(subset_comments_input, database_words_input):
    
    for dx_sx_single_value in dx_sx:
        
        print("Now dx/sx ---->  ", dx_sx_single_value)
        #skip the nan values
        if dx_sx_single_value == 'nan': continue
        
        subset_comments = subset_comments_input[subset_comments_input.loc[:,'p_dx_sx'] == str(dx_sx_single_value)]
        if len(subset_comments) == 0: continue
        
        database_words = database_words_input.copy()
        
        matching_indexes = database_words.index.intersection(subset_comments.index)
        subset_words = database_words.loc[matching_indexes,:]
        df_words = subset_words.copy()
        df_words['id']      = df_words.index
        df_words['c_rating'] = subset_comments.c_rating
        
        unique_words = {}
        words_per_comment = {}
        
        words_hate   = {}
        words_prob   = {}
        words_pos    = {}

        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                for single_word in row['text_nlp']:
                    unique_words.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                    unique_words[single_word] += 1

                    if row['c_rating'] == 'problematico':
                        words_prob.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_prob[single_word] += 1

                    elif row['c_rating'] == 'positivo':
                        words_pos.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_pos[single_word] += 1

                    elif row['c_rating'] == 'hate':
                        words_hate.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
                        words_hate[single_word] += 1

        unique_words.pop('#', None)
        
        lemmas_df         = pd.DataFrame.from_dict(unique_words, orient='index').dropna().reset_index()
        lemmas_df         = lemmas_df.reset_index()
        lemmas_df.columns = ['id', 'label', 'count']
        lemmas_df         = lemmas_df.drop('count', axis = 1)
        
        lemmas_df['counts_hate'] = lemmas_df.label.apply(lambda x: words_hate.get(x) if x in words_hate.keys() else 0)
        lemmas_df['counts_prob'] = lemmas_df.label.apply(lambda x: words_prob.get(x) if x in words_prob.keys() else 0)
        lemmas_df['counts_pos']  = lemmas_df.label.apply(lambda x: words_pos.get(x) if x in words_pos.keys() else 0)
        
        lemmas_df.to_csv('subsets/NODES_'+str(dx_sx_single_value)+'_hate_subset.csv', index = False)

        words_per_comment = {}

        for index, row in df_words.iterrows():
            if(len(row['text_nlp'])>0):
                words_per_comment[row["id"]]= [get_id_from_node(clean_txt, lemmas_df) for clean_txt in row['text_nlp']
                                               if clean_txt!='#'] ###iterating the column text_clean. There are 78174 rows


        clean_df = pd.DataFrame(0, index=lemmas_df.index, columns=lemmas_df.index)

        for key in words_per_comment:
            for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
                if pair[0]!=pair[1] and not(clean_df.at[pair[0],pair[1]]):
                    clean_df.at[pair[0],pair[1]] += 1
                    clean_df.at[pair[1],pair[0]] += 1
                    
                    
        words_projection = {} ##create dictionary 
        ##itertools.product() which computes the cartesian product of input iterables.

        for key in words_per_comment:
            for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
                if pair[0]!=pair[1] and not(pair[::-1] in words_projection):
                    words_projection.setdefault(pair,0)
                    words_projection[pair] += 1
                    
        words_weighted = []
        for edge in words_projection:
            words_weighted.append((edge[0],edge[1],words_projection[edge]))

        G = nx.Graph()
        G.add_weighted_edges_from(words_weighted)
        nx.write_weighted_edgelist(G, "subsets/EDGES_"+str(dx_sx_single_value)+"_hate_subset.csv",delimiter=",") ##save the edges list as csv
        df_edges = pd.read_csv('subsets/EDGES_'+str(dx_sx_single_value)+'_hate_subset.csv')
        df_edges.columns = ['source', 'target', 'weight']
        df_edges.to_csv('subsets/EDGES_'+str(dx_sx_single_value)+'_hate_subset.csv', index = False)
        
get_subset_dxsx(subset_comments, database_words)

Now dx/sx ---->   destra
Now dx/sx ---->   sinistra


## ADD SENTIMENT

In [7]:
filename_nodes = ['subsets/NODES_'+str(dx_sx_sing)+'_hate_subset.csv' for dx_sx_sing in dx_sx]
subset_DX = pd.read_csv(filename_nodes[0])
subset_DX = subset_DX.dropna().reset_index(drop = True)

subset_SX = pd.read_csv(filename_nodes[1])
subset_SX = subset_SX.dropna().reset_index(drop = True)
subset_SX

,id,label,counts_hate,counts_prob,counts_pos
0,0,giornalista,1,21,97
1,1,lecchina,0,2,0
2,2,salvare,0,13,84
3,3,bruciare,1,5,37
4,4,ano,0,1,1
...,...,...,...,...,...
29504,29505,ilegale,0,0,0
29505,29506,scappereccio,0,0,0
29506,29507,sconfiggerlo,0,0,0
29507,29508,luomo,0,0,0


In [8]:
sentix = pd.read_csv('lemmas_sentiment_correct.csv', sep = ",")
sentix

,lemma,positive_score,negative_score,polarity,intensity
0,diro,0.1250,0.28125,-0.467500,0.307777
1,gente,0.4375,0.18750,0.484476,0.475986
2,guerra,0.0625,0.12500,-0.409666,0.139754
3,portato,0.1250,0.43750,-0.645658,0.455007
4,casa,0.0650,0.17500,-0.547190,0.186682
...,...,...,...,...,...
40014,#rip,0.0000,0.00000,0.000000,0.000000
40015,#aspettandoprometeo,0.0000,0.00000,0.000000,0.000000
40016,#ivreich,0.0000,0.00000,0.000000,0.000000
40017,satanasso,0.0000,0.50000,-1.000000,0.500000


Add sentiment to dx database:

In [9]:
polarity_list = []

for i in range(len(subset_DX)):
    
    if (i%250 == 0): print(i, "/", len(subset_DX))
    
    word      = subset_DX.label[i]    
    polarity_list.append(sentix[sentix.loc[:,'lemma'] == word].polarity.tolist()[0])
polarity_list

0 / 23861
250 / 23861
500 / 23861
750 / 23861
1000 / 23861
1250 / 23861
1500 / 23861
1750 / 23861
2000 / 23861
2250 / 23861
2500 / 23861
2750 / 23861
3000 / 23861
3250 / 23861
3500 / 23861
3750 / 23861
4000 / 23861
4250 / 23861
4500 / 23861
4750 / 23861
5000 / 23861
5250 / 23861
5500 / 23861
5750 / 23861
6000 / 23861
6250 / 23861
6500 / 23861
6750 / 23861
7000 / 23861
7250 / 23861
7500 / 23861
7750 / 23861
8000 / 23861
8250 / 23861
8500 / 23861
8750 / 23861
9000 / 23861
9250 / 23861
9500 / 23861
9750 / 23861
10000 / 23861
10250 / 23861
10500 / 23861
10750 / 23861
11000 / 23861
11250 / 23861
11500 / 23861
11750 / 23861
12000 / 23861
12250 / 23861
12500 / 23861
12750 / 23861
13000 / 23861
13250 / 23861
13500 / 23861
13750 / 23861
14000 / 23861
14250 / 23861
14500 / 23861
14750 / 23861
15000 / 23861
15250 / 23861
15500 / 23861
15750 / 23861
16000 / 23861
16250 / 23861
16500 / 23861
16750 / 23861
17000 / 23861
17250 / 23861
17500 / 23861
17750 / 23861
18000 / 23861
18250 / 23861
18500 / 23

[-0.4675002450093739,
 0.4844757663633736,
 -0.40966552939826695,
 -0.6456578688683812,
 -0.5471903285814135,
 0.8193310587965338,
 -0.6456578688683812,
 0.9092085160005584,
 0.31191652150947724,
 0.0,
 -0.8791037326443221,
 0.7826131770709492,
 -0.4096655293982672,
 -1.0,
 0.4990430888876027,
 -1.0,
 -0.5155242336366264,
 1.0,
 0.0,
 1.0,
 -1.0,
 1.0,
 -0.8193310587965339,
 -0.4568010048796325,
 -1.0,
 -1.0,
 1.0,
 -0.8508925153831794,
 -1.0,
 0.0,
 1.0,
 0.7710034118051993,
 0.2513318327560048,
 1.0,
 -1.0,
 -1.0,
 0.0,
 -0.5903344706017328,
 0.0,
 0.0,
 -1.0,
 0.0,
 1.0,
 1.0,
 -1.0,
 -0.7057662241069511,
 0.7486681672439952,
 0.4844757663633736,
 -1.0,
 -0.40966552939826695,
 0.0,
 -0.5483747371186014,
 0.2513318327560048,
 -1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 -0.30156520855914426,
 -1.0,
 0.0,
 -1.0,
 1.0,
 -1.0,
 0.7215820509077442,
 0.21027382684501386,
 -0.5903344706017328,
 -0.3337413473589035,
 0.40966552939826695,
 1.0,
 0.8193310587965338,
 0.0,
 1.0,
 1.0,
 -1.0,
 0.590334470601

In [10]:
subset_DX['polarity'] = polarity_list
subset_DX.head(10)

,id,label,counts_hate,counts_prob,counts_pos,polarity
0,0,diro,1,65,218,-0.467500
1,1,gente,15,99,232,0.484476
2,2,guerra,2,65,101,-0.409666
3,3,portato,3,12,11,-0.645658
4,4,casa,27,219,193,-0.547190
5,5,rimpatriare,2,18,10,0.819331
6,6,marcello,0,1,117,-0.645658
7,7,perfavore,0,1,1,0.909209
8,8,patrio,0,25,54,0.311917
9,9,patriota,0,5,9,0.000000


In [11]:
subset_DX.to_csv(filename_nodes[0], index = False)

In [13]:
polarity_list = []

for i in range(len(subset_SX)):
    
    if (i%250 == 0): print(i, "/", len(subset_SX))
    
    word      = subset_SX.label[i]    
    polarity_list.append(sentix[sentix.loc[:,'lemma'] == word].polarity.tolist()[0])
polarity_list

0 / 29509
250 / 29509
500 / 29509
750 / 29509
1000 / 29509
1250 / 29509
1500 / 29509
1750 / 29509
2000 / 29509
2250 / 29509
2500 / 29509
2750 / 29509
3000 / 29509
3250 / 29509
3500 / 29509
3750 / 29509
4000 / 29509
4250 / 29509
4500 / 29509
4750 / 29509
5000 / 29509
5250 / 29509
5500 / 29509
5750 / 29509
6000 / 29509
6250 / 29509
6500 / 29509
6750 / 29509
7000 / 29509
7250 / 29509
7500 / 29509
7750 / 29509
8000 / 29509
8250 / 29509
8500 / 29509
8750 / 29509
9000 / 29509
9250 / 29509
9500 / 29509
9750 / 29509
10000 / 29509
10250 / 29509
10500 / 29509
10750 / 29509
11000 / 29509
11250 / 29509
11500 / 29509
11750 / 29509
12000 / 29509
12250 / 29509
12500 / 29509
12750 / 29509
13000 / 29509
13250 / 29509
13500 / 29509
13750 / 29509
14000 / 29509
14250 / 29509
14500 / 29509
14750 / 29509
15000 / 29509
15250 / 29509
15500 / 29509
15750 / 29509
16000 / 29509
16250 / 29509
16500 / 29509
16750 / 29509
17000 / 29509
17250 / 29509
17500 / 29509
17750 / 29509
18000 / 29509
18250 / 29509
18500 / 29

[-1.0,
 0.2513318327560048,
 1.0,
 -1.0,
 -0.6447133293200356,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -0.8085411514630199,
 0.6456578688683811,
 -0.6289056836890308,
 1.0,
 1.0,
 -0.6357879925770047,
 -1.0,
 0.31191652150947724,
 -1.0,
 -0.21027382684501367,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -0.911313042257116,
 -1.0,
 -1.0,
 0.0,
 0.7486681672439952,
 1.0,
 -0.7897261731549865,
 -1.0,
 1.0,
 0.14089314990910895,
 1.0,
 1.0,
 0.0,
 1.0,
 0.8193310587965338,
 1.0,
 -1.0,
 0.3229878906786693,
 0.0,
 -1.0,
 0.563248354265482,
 1.0,
 -1.0,
 0.0,
 0.5903344706017328,
 -0.40966552939826695,
 -1.0,
 0.4844757663633736,
 0.23314393360736885,
 0.31191652150947713,
 -1.0,
 0.0,
 -1.0,
 0.5903344706017332,
 1.0,
 1.0,
 0.40966552939826695,
 0.6357879925770048,
 -0.5155242336366264,
 -1.0,
 -0.436751645734518,
 -0.5903344706017328,
 1.0,
 -1.0,
 1.0,
 -0.1687698526353656,
 1.0,
 1.0,
 0.18066894120346624,
 -0.40966552939826695,
 0.6456578688683811,
 0.40966552939826695,
 -1.0,
 

In [15]:
subset_SX['polarity'] = polarity_list
subset_SX.to_csv(filename_nodes[1], index = False)

## LIWC part

In [70]:
subset_SX = pd.read_csv('subsets/NODES_sinistra_hate_subset.csv')
subset_DX = pd.read_csv('subsets/NODES_destra_hate_subset.csv')
subset_SX.head()

,id,label,counts_hate,counts_prob,counts_pos,polarity
0,0,giornalista,1,21,97,-1.000000
1,1,lecchina,0,2,0,0.251332
2,2,salvare,0,13,84,1.000000
3,3,bruciare,1,5,37,-1.000000
4,4,ano,0,1,1,-0.644713


In [71]:
#obtain the useful dataset
LIWC_dict = pd.read_csv('LIWC/LIWC2015 Results (Comments_lemmas_unique.csv).csv')
LIWC_dict = LIWC_dict.loc[1::,:].drop('A', axis = 1).dropna()
LIWC_dict

,B,WC,WPS,Sixltr,Dic,pronomi,Io,Noi,Se,Tu,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
1,diro,1,1.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,gente,1,1.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,guerra,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,portato,1,1.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,casa,1,1.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40016,#rip,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
40017,#aspettandoprometeo,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
40018,#ivreich,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
40019,satanasso,1,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
#list the variables one may want to keep
columns_to_keep = ['B', 'Emo_Neg',  'Ansia',  'Rabbia',  'Tristez', 'parolac']
att_to_keep = ['Emo_Neg',  'Ansia',  'Rabbia',  'Tristez', 'parolac']

In [73]:
LIWC_dict_sub = LIWC_dict.loc[:,columns_to_keep].dropna()
LIWC_dict_sub

,B,Emo_Neg,Ansia,Rabbia,Tristez,parolac
1,diro,0.0,0.0,0.0,0.0,0.0
2,gente,0.0,0.0,0.0,0.0,0.0
3,guerra,0.0,0.0,0.0,0.0,0.0
4,portato,0.0,0.0,0.0,0.0,0.0
5,casa,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
40016,#rip,0.0,0.0,0.0,0.0,0.0
40017,#aspettandoprometeo,0.0,0.0,0.0,0.0,0.0
40018,#ivreich,0.0,0.0,0.0,0.0,0.0
40019,satanasso,0.0,0.0,0.0,0.0,0.0


**Update left parties comments**

In [81]:
for i, word in enumerate(subset_SX.label):
    
    subsub = LIWC_dict_sub[LIWC_dict_sub.B == word].loc[:,att_to_keep]
    for j in att_to_keep: 
        subset_SX.loc[i, j] = subsub.loc[:,j].tolist()[0]
        
    if i%1500 == 0: print(i, "/", len(subset_SX.label))

print("Done.")
subset_SX.to_csv('subsets/NODES_sinistra.csv', index=False)
subset_SX

0 / 29509
1500 / 29509
3000 / 29509
4500 / 29509
6000 / 29509
7500 / 29509
9000 / 29509
10500 / 29509
12000 / 29509
13500 / 29509
15000 / 29509
16500 / 29509
18000 / 29509
19500 / 29509
21000 / 29509
22500 / 29509
24000 / 29509
25500 / 29509
27000 / 29509
28500 / 29509
Done.


,id,label,counts_hate,counts_prob,counts_pos,polarity,Emo_Neg,Ansia,Rabbia,Tristez,parolac
0,0,giornalista,1,21,97,-1.000000,0.0,0.0,0.0,0.0,0.0
1,1,lecchina,0,2,0,0.251332,0.0,0.0,0.0,0.0,0.0
2,2,salvare,0,13,84,1.000000,0.0,0.0,0.0,0.0,0.0
3,3,bruciare,1,5,37,-1.000000,0.0,0.0,0.0,0.0,0.0
4,4,ano,0,1,1,-0.644713,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
29504,29505,ilegale,0,0,0,-0.288769,0.0,0.0,0.0,0.0,0.0
29505,29506,scappereccio,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
29506,29507,sconfiggerlo,0,0,0,-1.000000,0.0,0.0,0.0,0.0,0.0
29507,29508,luomo,0,0,0,1.000000,0.0,0.0,0.0,0.0,0.0


**Update right parties comments**

In [82]:
for i, word in enumerate(subset_DX.label):
    
    subsub = LIWC_dict_sub[LIWC_dict_sub.B == word].loc[:,att_to_keep]
    for j in att_to_keep: 
        subset_DX.loc[i, j] = subsub.loc[:,j].tolist()[0]
        
    if i%1000 == 0: print(i, "/", len(subset_DX.label))
        
print("Done.")
subset_DX.to_csv('subsets/NODES_destra.csv', index=False)
subset_DX

0 / 23861
1000 / 23861
2000 / 23861
3000 / 23861
4000 / 23861
5000 / 23861
6000 / 23861
7000 / 23861
8000 / 23861
9000 / 23861
10000 / 23861
11000 / 23861
12000 / 23861
13000 / 23861
14000 / 23861
15000 / 23861
16000 / 23861
17000 / 23861
18000 / 23861
19000 / 23861
20000 / 23861
21000 / 23861
22000 / 23861
23000 / 23861
Done.


,id,label,counts_hate,counts_prob,counts_pos,polarity,Emo_Neg,Ansia,Rabbia,Tristez,parolac
0,0,diro,1,65,218,-0.467500,0.0,0.0,0.0,0.0,0.0
1,1,gente,15,99,232,0.484476,0.0,0.0,0.0,0.0,0.0
2,2,guerra,2,65,101,-0.409666,0.0,0.0,0.0,0.0,0.0
3,3,portato,3,12,11,-0.645658,0.0,0.0,0.0,0.0,0.0
4,4,casa,27,219,193,-0.547190,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
23856,23857,satanasso,0,0,0,-1.000000,0.0,0.0,0.0,0.0,0.0
23857,23858,pezzettino,0,0,0,-1.000000,0.0,0.0,0.0,0.0,0.0
23858,23859,tendopoli,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
23859,23860,barbetta,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
